In [1]:
import pandas as pd
import numpy as np

df_origin = pd.read_csv('../backup/HN_cardinal_1_8.csv')
df_origin.shape

FileNotFoundError: [Errno 2] No such file or directory: '../backup/HN_cardinal_1_8.csv'

In [ ]:
#2020년 은 테스트셋으로 분리
df_train = df_origin[df_origin['year'] != 2020]
df_test = df_origin[df_origin['year'] == 2020]

In [ ]:
# 4~8기 19세 이상
df_train = df_train[(df_train['cardinal_number'] >= 4) & (df_train['cardinal_number'] <= 8) & (df_train['age'] >= 19)]
df_train.shape

(87986, 4025)

In [ ]:
df_test = df_test[(df_test['cardinal_number'] >= 4) & (df_test['cardinal_number'] <= 8) & (df_test['age'] >= 19)]
df_test.shape

(6133, 4025)

In [ ]:
num_features = ["age",
# "ainc", # Random Forest Importance Top 4
"HE_PLS_15",
"HE_PLS_30",
"HE_mPLS",
"HE_sbp",
"HE_dbp",
"HE_ht",
"HE_wt",
"HE_wc",
"HE_glu",
"HE_HbA1c",
"HE_insulin",
"HE_chol",
"HE_HDL_st2",
"HE_TG",
"HE_LDL_drct",
"HE_ast",
"HE_alt",
"HE_HB",
"HE_HCT",
"HE_BUN",
"HE_crea",
"HE_WBC",
"HE_RBC",
"HE_Bplt",
"HE_Uacid",
"HE_Uph",
"HE_Usg",
"HE_Ucrea"]

In [ ]:
onehot_features = ["region",
"sex",
"occp",
"marri_1",
"tins",
"npins",
"D_2_1",
"DI1_pr",
"DI1_pt",
"DI2_pr",
"DI2_pt",
"DI3_pr",
"DI3_pt",
"DJ4_pr",
"DJ4_pt",
"DE1_pr",
"DE1_pt",
"BH9_11",
"BH1",
"BH2_61",
"LQ4_00", # Random Forest Importance Top 3 여기서 제거할 수 없음 LQ4_00_1.0
"LQ1_sb",
"LQ2_ab",
"AC1_yr",
"MH1_yr",
"EC_occp",
"EC_stt_1",
"EC_stt_2",
"EC_wht_0",
"BO1_1",
"BO2_1",
"BD1",
"BS8_2",
"BS9_2", # Random Forest Importance Top 2 여기서 제거할 수 없음 BS9_2_8.0
"LW_ms",
"LW_pr",
"LW_oc",
"HE_rPLS",
"HE_Unitr",
"HE_Upro",
"HE_Uglu",
"HE_Uket",
"HE_Ubil",
"HE_Ubld",
"HE_Uro",
"BM1_0",
"BM1_1",
"BM1_2",
"BM1_3",
"BM1_4",
"BM1_5",
"BM1_6",
"BM1_7",
"BM1_8",
"L_BR_TO",
"L_LN_TO",
"L_DN_TO",
"LK_LB_US",
"LK_LB_EF",
"live_t",
"educ",
"BO1",
"HE_obe",]

In [ ]:
label_features = ["incm",
"ho_incm",
"incm5",
"ho_incm5",
"edu",
"cfam",
"house",
"D_1_1", # Random Forest Importance Top 1
"DI3_2",
"BD1_11",
"BD2_1",
"BA2_12",
"BA2_13",
# "BP1",
"BS3_1",
"BE3_31",
"BE5_1",
"OR1",
]

In [ ]:
y_related =["BP_PHQ_1",
"BP_PHQ_2",
"BP_PHQ_3",
"BP_PHQ_4",
"BP_PHQ_5",
"BP_PHQ_6",
"BP_PHQ_7",
"BP_PHQ_8",
"BP_PHQ_9",
"mh_PHQ_S",
"BP6_10",
"BP6_31",
"DF2_pr",
"DF2_pt", "BP1"]

In [ ]:
df_train = df_train[num_features + onehot_features + label_features + y_related]
df_test = df_test[num_features + onehot_features + label_features + y_related]

In [ ]:
def manual_preprocess(df:pd.DataFrame, num_features, onehot_features, label_features, y_related):
    # 1) 9, 99, 999 인 녀석들(모름/무응답) null 처리
    df = df.replace([9, 99, 999, 9999], np.nan, inplace=False)

    # 2) mh_PHQ_S 는 null값인데, 각 문항 응답인 BP_PHQ_1 ~ 9의 값이 있는 경우, 더해서 mh_PHQ_S에 넣어주기
    cond = df["BP_PHQ_1"].notna() & \
            df["BP_PHQ_2"].notna() & \
            df["BP_PHQ_3"].notna() & \
            df["BP_PHQ_4"].notna() & \
            df["BP_PHQ_5"].notna() & \
            df["BP_PHQ_6"].notna() & \
            df["BP_PHQ_7"].notna() & \
            df["BP_PHQ_8"].notna() & \
            df["BP_PHQ_9"].notna()
    
    a = df[df["mh_PHQ_S"].isnull() & cond][["BP_PHQ_1", "BP_PHQ_2", "BP_PHQ_3", "BP_PHQ_4", "BP_PHQ_5", "BP_PHQ_6", "BP_PHQ_7", "BP_PHQ_8", "BP_PHQ_9"]]
    b = a[a <= 3].sum(axis=1)
    b[b >= 1] #125

    df['mh_PHQ_S'] = b[b >=1]

    # 3) 결측값 처리 1 - row 기준
    threshold = 0.3 # threshold for null proportion
    null_percent = df.isnull().mean(axis = 1) # 각 행에 대한 결측값 비율 계산
    df = df[null_percent < threshold]
    
    # 4) 변수 생성 `depressed` 
    cond1 = df['mh_PHQ_S'] >= 10 # mh_PHQ_S - total score 
    cond2 = df['BP_PHQ_9'].isin([1, 2, 3]) # person who chose 1, 2, 3 in 9th question 
    cond3 = df['BP6_10'] == 1 
    cond4 = df['BP6_31'] == 1
    cond5 = (df['DF2_pr'] == 1) & (df['mh_PHQ_S'].isna())

    df[cond1 | cond2 | cond3 | cond4 | cond5].drop_duplicates()
 
    df.loc[cond1 | cond2 | cond3 | cond4 | cond5, 'depressed'] = 1 # 조건 중 하나 이상을 만족하는 행에 'depressed' 변수에 1 값 
    df.loc[~(cond1 | cond2 | cond3 | cond4 | cond5), 'depressed'] = 0 # 조건 중 하나라도 만족하지 않는 행에 0 값

    # 5) 학습에 방해가 될 수 있는 그룹 제거 (우울증에 관한 정보가 없는 경우)
    df.drop(df[df['DF2_pr'].isna() & df['mh_PHQ_S'].isna()].index, inplace=True)
    df = df[(df['DF2_pr'] != 8 |  ~df['mh_PHQ_S'].isna())]

    # 6) 결측값 처리 2 - columns 기준
    threshold = 0.1 # 10% 기준
    null_percent = df.isnull().mean() # 각 열에 대한 결측값 비율 계산
    selected_cols = null_percent[null_percent < threshold].index
    df = df[selected_cols]

    # 7) y_related 변수를 피쳐에서 제외하기
    remained_y = set(df.columns).intersection(set(y_related))
    y = df['depressed'] # y 생성
    X = df.drop(columns=list(remained_y), inplace=False) # X 생성

    # 8) 위의 결측값 처리에서 빠진 column을 각 피쳐 그룹에서 제외하기
    num_features = set(df.columns).intersection(set(num_features))
    onehot_features = set(df.columns).intersection(set(onehot_features))
    label_features = set(df.columns).intersection(set(label_features))

    # 9) label features중 비해당(8) 값을 각각 알맞은 값으로 대체하기
    label_ans_8 = ["DI3_2", "BD1_11", "BD2_11", "BA2_12", "BA2_13", "BP1", "BS3_1","BE3_31", "BE5_1"] # BD2_11 - removed

    # 'DI3_2' 변수의 값이 8인 경우, 해당 값을 0으로 바꾸기
    X['DI3_2'] = X['DI3_2'].apply(lambda x: 3 if x == 8 else x)
    X['BD1_11'] = X['BD1_11'].apply(lambda x: 1 if x == 8 else x)
    # X['BD2_11'] = X['BD2_11'].apply(lambda x: 0 if x == 8 else x)
    X['BA2_12'] = X['BA2_12'].apply(lambda x: 5 if x == 8 else x)
    X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
    X['BA2_13'] = X['BA2_13'].apply(lambda x: 5 if x == 8 else x)
    # X['BP1'] = X['BP1'].apply(lambda x: 5 if x == 8 else x)
    X['BS3_1'] = X['BS3_1'].apply(lambda x: 0 if x == 8 else x)
    # X['BE3_31'] = X['BE3_31'].apply(lambda x: 5 if x == 88 else x)
    # X['BE5_1'] = X['BE5_1'].apply(lambda x: 5 if x == 88 else x)

    print("X.shape:", X.shape)
    print("y.shape:", y.shape)

    return X, y, list(num_features), list(onehot_features), list(label_features), list(y_related)

In [ ]:
trn_X, trn_y, num_features, onehot_features, label_features, y_related = manual_preprocess(df_train, num_features, onehot_features, label_features, y_related)
tst_X, tst_y, num_features, onehot_features, label_features, y_related = manual_preprocess(df_test, num_features, onehot_features, label_features, y_related)

X.shape: (74517, 89)
y.shape: (74517,)
X.shape: (5226, 99)
y.shape: (5226,)


In [ ]:
trn_X.shape, tst_X.shape

((74517, 89), (5226, 99))

In [13]:
tst_X = tst_X[list(trn_X.columns)]
tst_X.shape

(5226, 89)

In [14]:
trn_X.to_csv('../data/trn_X_20231211_final_col_01.csv', index=False)
trn_y.to_csv('../data/trn_y_20231211_final_col_01.csv', index=False)

tst_X.to_csv('../data/tst_X_20231211_final_col_01.csv', index=False)
tst_y.to_csv('../data/tst_y_20231211_final_col_01.csv', index=False)

In [15]:
print(num_features)
print(onehot_features)
print(label_features)
print(y_related)

['HE_glu', 'HE_ast', 'HE_WBC', 'HE_Bplt', 'HE_dbp', 'ainc', 'HE_HDL_st2', 'HE_sbp', 'HE_wt', 'HE_HB', 'HE_chol', 'HE_RBC', 'HE_Usg', 'HE_alt', 'HE_wc', 'HE_crea', 'HE_Uph', 'HE_ht', 'HE_HCT', 'HE_BUN', 'age', 'HE_TG']
['npins', 'DI2_pr', 'DJ4_pr', 'HE_Ubil', 'educ', 'BH1', 'sex', 'BO1', 'marri_1', 'HE_obe', 'DI2_pt', 'live_t', 'tins', 'HE_Uro', 'DI3_pr', 'BH9_11', 'EC_stt_1', 'occp', 'HE_Unitr', 'BO1_1', 'LQ4_00', 'HE_Upro', 'DE1_pr', 'DE1_pt', 'HE_Uglu', 'BM1_4', 'BM1_6', 'HE_rPLS', 'LQ1_sb', 'D_2_1', 'BH2_61', 'HE_Uket', 'BM1_8', 'BO2_1', 'EC_stt_2', 'BM1_3', 'DJ4_pt', 'HE_Ubld', 'BM1_7', 'BS8_2', 'LQ2_ab', 'EC_occp', 'BS9_2', 'BM1_5', 'DI3_pt', 'BD1', 'BM1_1', 'DI1_pr', 'DI1_pt', 'BM1_2']
['cfam', 'BA2_13', 'BE5_1', 'BD2_1', 'DI3_2', 'edu', 'BD1_11', 'house', 'BE3_31', 'BA2_12', 'ho_incm5', 'incm', 'ho_incm', 'incm5', 'BS3_1', 'D_1_1']
['BP_PHQ_1', 'BP_PHQ_2', 'BP_PHQ_3', 'BP_PHQ_4', 'BP_PHQ_5', 'BP_PHQ_6', 'BP_PHQ_7', 'BP_PHQ_8', 'BP_PHQ_9', 'mh_PHQ_S', 'BP6_10', 'BP6_31', 'DF2_pr'